In [ ]:
import torch
from PIL import Image
import open_clip
import os

In [ ]:
name = "ViT-B-16-plus-240"
pretrained = "laion400m_e32"
model, train_transform, eval_transform = open_clip.create_model_and_transforms(name, pretrained=pretrained)

In [ ]:
from pathlib import Path

ouptut_dir = "openvino_fp32"
if not os.path.exists(ouptut_dir):
    os.makedirs(ouptut_dir)
    
with open(Path(ouptut_dir) / "model_index.txt", 'w') as fd:
    fd.write(f"{name},{pretrained}\n")

In [ ]:
tokenizer = open_clip.get_tokenizer('ViT-B-16-plus-240')

image = eval_transform(Image.open("../docs/CLIP.png")).unsqueeze(0)
text = tokenizer("a cat")

In [ ]:
torch.onnx.export(
    model.visual,
    image,
    "image_encoder.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["image"],  # the model's input names
    output_names=["image_embedding"],  # the model's output names
    dynamic_axes={  # variable length axes
        "image": {0: "batch"},
        "image_embedding": {0: "batch"},
    }
)

In [ ]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_encoder = convert_model("image_encoder.onnx")
ov.serialize(ov_encoder, ouptut_dir + "/image_encoder.xml")

In [ ]:
class TextTransformerExportWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, text):
        x = self.model.token_embedding(text)
        x = x + self.model.positional_embedding
        x = x.permute(1, 0, 2)  # NLD -> LND
        x = self.model.transformer(x, attn_mask=self.model.attn_mask)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.model.ln_final(x)  # [batch_size, n_ctx, transformer.width]
        # take features from the eot embedding (eot_token is the highest number in each sequence)
        x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ self.model.text_projection
        return x

transformer_export_wrapper = TextTransformerExportWrapper(model)

In [ ]:
torch.onnx.export(
    transformer_export_wrapper,
    text,
    "text_encoder.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["input_ids"],  # the model's input names
    output_names=["text_embeds"],  # the model's output names
    dynamic_axes={  # variable length axes
        "input_ids": {0: "batch"}
    }
)

In [ ]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_transformer = convert_model("text_encoder.onnx")
ov.serialize(ov_transformer, ouptut_dir + "/text_encoder.xml")

In [ ]:
!benchmark_app -m image_encoder.onnx -shape "image[1,3,240,240]" -api sync

In [ ]:
!benchmark_app -m text_encoder.onnx -shape "input_ids[1,77]" -api sync

In [ ]:
torch.onnx.export(
    model,
    (image, text),
    "model.onnx",  # where to save the model
    opset_version=14,  # the ONNX version to export the model to
    input_names=["image", "text"],  # the model's input names
    output_names=["image_embedding"],  # the model's output names
    dynamic_axes={  # variable length axes
        "image": {0: "batch"},
        "text": {0: "batch"},
    }
)

In [ ]:
from openvino.tools.mo import convert_model
import openvino.runtime as ov

ov_transformer = convert_model("model.onnx")
ov.serialize(ov_transformer, "model.xml")

In [ ]:
!benchmark_app -m model.xml -shape "image[1,3,240,240],text[1,77]" -api sync